# [LVV-T2216] Integrated Slew and Tracking Test

This notebook is used to execute the [LVV-T2216 (1.0)] test script during System Spread Integration Tests on Level 3.  
Execution steps are separated by horizontal lines.   
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA.  

**Note:**
 - [LVV-T2216] is similar to [LVV-T2290 (2.0)] test case, but without taking a ComCam image.  
   The other difference is that M1M3 and M2 are using mount data instead of their internal inclinometer.  
   Because of this, make sure that you move the telescope in small steps and always between 82.5 deg and 86.5 deg.
   
**Requirements:**
 - All the MT components should be enabled.
 
Execution steps are separated by horizontal lines.  
Upon completion, save the notebook and its output as a pdf file to be attached to the test execution in JIRA. 

**Make sure you run this notebook on TTS before running at the summit.**

Please, see the [README] file for the requirements to run this notebook.

[LVV-T2215 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2215
[LVV-T2216]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2216
[LVV-T2290 (2.0)]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2290  
[README]: https://github.com/lsst-sitcom/notebooks_vandv/blob/develop/README.md

## Setting Up Test Environment

Before we run the tests, we want to make sure that we have all the libraries imported, remotes connected, etc.

In [ ]:
test_case = "LVV-T2216"
test_exec = "LVV-EXXXX"

In [ ]:
%load_ext autoreload
%autoreload 2

import os
import sys
import asyncio
import logging
import time

import pandas as pd
import numpy as np

from matplotlib import pyplot as plt
from astropy.time import Time
from datetime import datetime, timedelta
import pandas as pd

from lsst.ts import salobj
from lsst.ts.observatory.control.maintel import MTCS, ComCam
from lsst.ts.observatory.control import RotType

from lsst.sitcom import vandv

In [ ]:
exec_info = vandv.ExecutionInfo()
print(exec_info)

Use the [LVV-T2344] to test case and notebook to setup all the main telescope components.  
This includes simulators as well as real hardware when available (this will depend on when the test is conducted at TTS or on level 3 or on the telescope):  

- pointing  
- mount ( with the CCW)  
- rotator  
- ready M1M3: raise mirror, turn on FB, clear forces. Note that if used at level 3, we need to have M1M3 LUT use mount telemetry  
- ready M2: turn on FB, clear forces. Note that if used at level 3, we need to have M2 LUT use mount telemetry  
- Get cam hex Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Get M2 hex (simulator) Ready: check config; make sure LUT is on and has valid inputs; make sure hex is at LUT position  
- Finally, get the MTAOS CSC ready  

[LVV-T2344]: https://jira.lsstcorp.org/secure/Tests.jspa#/testCase/LVV-T2344

In [ ]:
logging.basicConfig(format="%(asctime)s %(name)s: %(message)s", level=logging.DEBUG)

In [ ]:
log = logging.getLogger("setup")
log.level = logging.DEBUG

In [ ]:
os.environ["LSST_DDS_HISTORYSYNC"] = "200"
domain = salobj.Domain()

In [ ]:
mtcs = MTCS(domain=domain, log=log)
mtcs.set_rem_loglevel(logging.ERROR)

In [ ]:
await mtcs.start_task

The cell below exposes the mount so we have a more compact code when moving the telescope carefully while M1M3 and M2 are using it to calculate their look-up tables instead of using their internal inclinometer.

In [ ]:
mtmount = mtcs.rem.mtmount

In [ ]:
script = salobj.Controller("Script", index=42658885)
await asyncio.sleep(10) 

script.start_task

## Slewing without LUT corrections
  
Keep M1M3 and M2 subscribing to their own inclinometer for this step.  
  
Do the following slew sequence, and watch the Chronograph to make sure there is no abrupt change in elevation angle (otherwise it would fault M1M3). 
This is done in anticipation of switching the M1M3 LUT from inclinometer to mount telemetry mode.  
  
Verify that the LUT corrections are not applied by looking at coronograph and ensuring the forces values don't change.  
For the hexapod we compare the compensated and uncompensated values.  
  
Do 4 slews in sequence.  
For each slew, track for 39s, simulating a visit.  
  
**Note:** 
- The 4 slews need to correspond to an elevation angle between 86.5 deg and 82 deg. 
- An additional margin is needed due to tracking.  
- [85.4, 84.4,83.4,82.4]  
  
Move to zenith at the end (so that we can start M1M3 with LUT in mount telemetry mode)

In [ ]:
await vandv.mount.moveMountInElevationSteps(mtmount, target_el=90, azimuth=0)

In [ ]:
# Ensure that we are not tracking
await mtcs.stop_tracking()

# Make sure M1M3 is raised
await mtcs.raise_m1m3()

# Make sure hardpoint corrections are enabled for M1M3
await mtcs.enable_m1m3_balance_system()
time.sleep(5)

await mtcs.reset_m1m3_forces()
time.sleep(5)

# Make sure hardpoint corrections are enabled for M2
await mtcs.enable_m2_balance_system()
time.sleep(5)

await mtcs.reset_m2_forces()
time.sleep(5)

In [ ]:
print(datetime.now())
script.log.info(f"START - {test_case} {test_exec} Slewing without LUT")

In [ ]:
target = mtcs.radec_from_azel(az=0, el=85.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=84.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=83.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=82.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
print(datetime.now())
script.log.info(f"END - {test_case} {test_exec} Slewing without LUT")

## Setting up components

### Get M1M3 LUT subscribed to the Mount

- Put M1M3 in STANDBY
- Change the configuration files to make M1M3 LUT to look at the mount elevation instead of the inclinometer.
- Reset the M1M3 CSC
- Put M1M3 back in ENABLED state
- Raise the mirror, turn on FB, clear forces.
- Check that the M1M3 LUT uses the mount telemetry.

In [ ]:
await mtcs.lower_m1m3()

await mtcs.set_state(
    state=salobj.State.STANDBY,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

<div class="alert alert-warning"> Change configuration. </div>

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm1m3"], 
    overrides = {"mtm1m3": 'Default'}
    )

In [ ]:
# Ensure that we are not tracking
await mtcs.stop_tracking()

# Make sure M1M3 is raised
await mtcs.raise_m1m3()

# Make sure hardpoint corrections are enabled for M1M3
await mtcs.enable_m1m3_balance_system()
time.sleep(5)

await mtcs.reset_m1m3_forces()
time.sleep(5)

### Get M2 LUT subscribed to the Mount

- Put M2 in STANDBY
- Change the configuration files to make M2 LUT look at the mount elevation instead of the inclinometer.
- Reset the M2 EUI.
- Put M2 back in ENABLED state
- Turn on FB, and clear forces.

In [ ]:
await mtcs.set_state(
    state=salobj.State.STANDBY,
    components=["mtm2"], 
    )

<div class="alert alert-warning"> Change configuration. </div>

In [ ]:
await mtcs.set_state(
    state=salobj.State.ENABLED,
    components=["mtm2"], 
    )

In [ ]:
# Make sure hardpoint corrections are enabled for M2
await mtcs.enable_m2_balance_system()
time.sleep(5)

await mtcs.reset_m2_forces()
time.sleep(5)

### Get CamHex ready
- Check config
- make sure LUT is on and has valid inputs
- make sure the Camhex is at LUT position

### Get M2 hex Ready
- Check config
- Make sure LUT is on and has valid inputs
- Make sure hex is at LUT position

## Slewing with LUT corrections using mount telemetry

Do 4 slews in sequence.  
For each slew, track for 39s, simulating a visit.  

**Note:** 
- the 4 slews need to correspond to an elevation angle between 86.5 deg and 82 deg.  
- An additional margin is needed due to tracking.  
- [85.4, 84.4,83.4,82.4]

In [ ]:
await vandv.mount.moveMountInElevationSteps(mtmount, target_el=90, azimuth=0)

In [ ]:
# Ensure that we are not tracking
await mtcs.stop_tracking()

# Make sure M1M3 is raised
await mtcs.raise_m1m3()

# Make sure hardpoint corrections are enabled for M1M3
await mtcs.enable_m1m3_balance_system()
time.sleep(5)

await mtcs.reset_m1m3_forces()
time.sleep(5)

# Make sure hardpoint corrections are enabled for M2
await mtcs.enable_m2_balance_system()
time.sleep(5)

await mtcs.reset_m2_forces()
time.sleep(5)

In [ ]:
await vandv.mount.moveMountInElevationSteps(mtmount, target_el=80, azimuth=0)

In [ ]:
print(datetime.now())
script.log.info(f"START - {test_case} {test_exec} Slewing with LUT")

In [ ]:
target = mtcs.radec_from_azel(az=0, el=85.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=84.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=83.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
target = mtcs.radec_from_azel(az=0, el=82.4)
await mtcs.slew_icrs(ra=target.ra, dec=target.dec, rot_type=RotType.Physical, rot=0)

time.sleep(39.)

In [ ]:
await mtcs.stop_tracking()

In [ ]:
print(datetime.now())
script.log.info(f"END - {test_case} {test_exec} Slewing with LUT")

***
## Plot The Results

Use the [LVV-T2216-plots] notebook to plot the results. 

[LVV-T2216-plots]: https://

***
## Wrap Up and Shut Down

This cell is not currently included as part of the test execution, but included here as needed to shutdown the systems

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtaos"])

In [ ]:
await mtcs.lower_m1m3()

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm1m3"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mtm2"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_1"])

In [ ]:
await mtcs.set_state(salobj.State.STANDBY, components=["mthexapod_2"])

In [ ]:
await mtcs.standby()